In [18]:
TOKEN = "sk-l7OhTietleKeiULzFkisT9BlbkFJdVZSOwC9PSt7HA5UmOzY"

# 互联网上找一个pdf文件，这里就用了openai官网的gpt-4的文档作为示例。用requests库，将pdf文件下载下来。

In [3]:
import requests
res = requests.get('https://cdn.openai.com/papers/gpt-4.pdf')
with open('gpt-4.pdf', 'wb') as f:
  f.write(res.content)

# 使用pypdf库获取pdf文件里的文字内容

In [4]:
! pip install pypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 KB 10.7 MB/s eta 0:00:00


In [5]:
from pypdf import PdfReader

reader = PdfReader("gpt-4.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [6]:
text

'GPT-4 Technical Report\nOpenAI\x03\nAbstract\nWe report the development of GPT-4, a large-scale, multimodal model which can\naccept image and text inputs and produce text outputs. While less capable than\nhumans in many real-world scenarios, GPT-4 exhibits human-level performance\non various professional and academic benchmarks, including passing a simulated\nbar exam with a score around the top 10% of test takers. GPT-4 is a Transformer-\nbased model pre-trained to predict the next token in a document. The post-training\nalignment process results in improved performance on measures of factuality and\nadherence to desired behavior. A core component of this project was developing\ninfrastructure and optimization methods that behave predictably across a wide\nrange of scales. This allowed us to accurately predict some aspects of GPT-4’s\nperformance based on models trained with no more than 1/1,000th the compute of\nGPT-4.\n1 Introduction\nThis technical report presents GPT-4, a large m

# 本来这里就可以调用GPT API进行处理了，但是GPT有单词输入文字的最大限制。所以要对pdf里的文字切分。等比例切分会中断句子。这里用了自然语言处理NLTK库，按句子的意思进行切分。

In [7]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
sentences = sent_tokenize(text)

In [15]:
for sentence in sentences:
    print(sentence)
    print('=' * 20)

GPT-4 Technical Report
OpenAI
Abstract
We report the development of GPT-4, a large-scale, multimodal model which can
accept image and text inputs and produce text outputs.
While less capable than
humans in many real-world scenarios, GPT-4 exhibits human-level performance
on various professional and academic benchmarks, including passing a simulated
bar exam with a score around the top 10% of test takers.
GPT-4 is a Transformer-
based model pre-trained to predict the next token in a document.
The post-training
alignment process results in improved performance on measures of factuality and
adherence to desired behavior.
A core component of this project was developing
infrastructure and optimization methods that behave predictably across a wide
range of scales.
This allowed us to accurately predict some aspects of GPT-4’s
performance based on models trained with no more than 1/1,000th the compute of
GPT-4.
1 Introduction
This technical report presents GPT-4, a large multimodal model capa

# 调用GPT API对分段的文字进行翻译

In [16]:
! pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.1 MB/s eta 0:00:00


In [25]:
import openai
openai.api_key = TOKEN

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "请你成为文章翻译的小帮手，请协助翻译以下技术文件，以简体中文输出"},
        {"role": "user", "content": sentences[0]},
    ]
)
completion.choices[0].message.content

'GPT-4技术报告\nOpenAI\n摘要\n本文报告了GPT-4的开发情况，这是一个大规模的、多模态模型，能够接受图像和文本输入，并生成文本输出。'

In [26]:
input_sentences = ''
chunks = []
for sentence in sentences:
  input_sentences += sentence
  if len(input_sentences) > 1000:
    chunks.append(input_sentences)
    input_sentences = ''
chunks.append(input_sentences)
chunks

['GPT-4 Technical Report\nOpenAI\x03\nAbstract\nWe report the development of GPT-4, a large-scale, multimodal model which can\naccept image and text inputs and produce text outputs.While less capable than\nhumans in many real-world scenarios, GPT-4 exhibits human-level performance\non various professional and academic benchmarks, including passing a simulated\nbar exam with a score around the top 10% of test takers.GPT-4 is a Transformer-\nbased model pre-trained to predict the next token in a document.The post-training\nalignment process results in improved performance on measures of factuality and\nadherence to desired behavior.A core component of this project was developing\ninfrastructure and optimization methods that behave predictably across a wide\nrange of scales.This allowed us to accurately predict some aspects of GPT-4’s\nperformance based on models trained with no more than 1/1,000th the compute of\nGPT-4.1 Introduction\nThis technical report presents GPT-4, a large multimo

In [27]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "请你成为文章翻译的小帮手，请协助翻译以下技术文件，以简体中文输出"},
        {"role": "user", "content": chunks[0]},
    ]
)
completion.choices[0].message.content

'GPT-4技术报告\nOpenAI\n摘要\n本文报道了GPT-4的开发情况，它是一个可接收图像和文本输入并生成文本输出的大规模多模态模型。虽然在许多实际情境中比人类能力差，但在各种专业和学术基准测试中，GPT-4表现出人类水平的性能，包括在模拟的律师考试中获得了约是前10%考生的成绩。GPT-4是基于Transformer的模型，预先训练以预测文档中的下一个标记。后训练对齐过程提高了其实际性能和符合所需行为的程度。该项目的核心组件是开发基础设施和优化方法，可在各种规模上可靠地预测GPT-4的某些方面性能，其中包括了通过使用不超过GPT-4计算量的1/1000的模型进行训练。\n1介绍\n本技术报告介绍了GPT-4，它是一个大规模多模态模型，能够处理图像和文本输入并生成文本输出。'

# 下面的代码将上面的所有分片的代码拼在在一起，完成对完整的pdf的翻译。

In [28]:
from pypdf import PdfReader
from nltk.tokenize import sent_tokenize

pdf_name = "gpt-4.pdf" 
reader = PdfReader(pdf_name)
number_of_pages = len(reader.pages)

chunks = []

for i in range(number_of_pages):
  page = reader.pages[i]
  text = page.extract_text()
  sentences = sent_tokenize(text)
  input_sentences = ''
  
  for sentence in sentences:
    input_sentences += sentence
    if len(input_sentences) > 1000:
      chunks.append(input_sentences)
      input_sentences = ''
  chunks.append(input_sentences)

for i in range(10):
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "请你成为文章翻译的小帮手，请协助翻译以下技术文件，以简体中文输出"},
        {"role": "user", "content": chunks[i]},
    ]
  )
  print('原文:', chunks[i])
  print('翻译结果:',completion.choices[0].message.content)

原文: GPT-4 Technical Report
OpenAI
Abstract
We report the development of GPT-4, a large-scale, multimodal model which can
accept image and text inputs and produce text outputs.While less capable than
humans in many real-world scenarios, GPT-4 exhibits human-level performance
on various professional and academic benchmarks, including passing a simulated
bar exam with a score around the top 10% of test takers.GPT-4 is a Transformer-
based model pre-trained to predict the next token in a document.The post-training
alignment process results in improved performance on measures of factuality and
adherence to desired behavior.A core component of this project was developing
infrastructure and optimization methods that behave predictably across a wide
range of scales.This allowed us to accurately predict some aspects of GPT-4’s
performance based on models trained with no more than 1/1,000th the compute of
GPT-4.1 Introduction
This technical report presents GPT-4, a large multimodal model capabl